In [ ]:
from collections import defaultdict
import pandas as pd
import random
from collections import defaultdict
from datetime import datetime
from tqdm import tqdm

### データセット作成条件

In [ ]:
# 　使用データセット
dataset = 'ml-1m'
# ユーザ数の指定フラグ
is_user_num_const = True
user_num = 30
# item/userの固定フラグ
is_item_per_user_num_const = True
item_per_user = 7

In [ ]:
data_path = '{}/ratings.dat'.format(dataset)

In [ ]:
df = pd.read_table(data_path, sep="::", header=None,
                names=['user_id', 'item_id', 'rating', 'time_stamp'], index_col=False)

In [ ]:
# df = df[0:10000]

In [ ]:
df.head(5)

In [ ]:
users = df['user_id']
items = df['item_id']
times = df['time_stamp']

In [ ]:
# 指定したユーザ数にデータを整形
if is_user_num_const:
    print('is_user_num_const = {}'.format(is_user_num_const))
    
    unique_users = df['user_id'].unique() 
    master_user = random.sample(list(unique_users), user_num)
    ignore_user = []
    
    users_ = users
    items_ = items
    times_ = times
    
    users = []
    items = []
    times = []
    
    for u, i, t in tqdm(zip(users_, items_, times_)):
        if u in master_user:
            users.append(u)
            items.append(i)
            times.append(t)

In [ ]:
# item/userを一定に整形
if is_item_per_user_num_const:
    print('is_item_per_user_num_const = {}'.format(is_item_per_user_num_const))
    print('item/user = {}'.format(item_per_user))
    
    User = defaultdict(list)
    
    for u, i, t in zip(users, items, times):
        User[u].append([i,t])
        
    for u in User:
        items = User[u]
        if len(items) > item_per_user:
            items = items[0:item_per_user]
            User[u] = items
    
    users = []
    items = []
    times = []
    
    for u in User:
        for values in User[u]:
            users.append(u)
            items.append(values[0])
            times.append(values[1])

In [ ]:
# 評価件数カウント用のdict
countU = defaultdict(lambda: 0)

In [ ]:
for user, item, time in zip(users, items, times):
    countU[user] += 1

In [ ]:
usermap = dict()
usernum = 0
itemmap = dict()
itemnum = 0
User = dict()

In [ ]:
for user, item, time in zip(users, items, times):
    #  評価件数がitem_per_user以下のものは無視    
    if countU[user] < item_per_user :
        continue
        
    if user in usermap:
        userid = usermap[user]
    else:
        usernum +=1
        userid = usernum
        usermap[user] = userid
        User[userid] = []
        
    if item in itemmap:
        itemid = itemmap[item]
    else:
        itemnum += 1
        itemid = itemnum
        itemmap[item] = itemid
        
    User[userid].append([time, itemid])

In [ ]:
# 時系列順にソートする
for userid in User.keys():
    User[userid].sort(key=lambda x: x[0])

In [ ]:
print (usernum, itemnum)

In [ ]:
usermap

In [ ]:
itemmap

In [ ]:
user_list = []
item_list = []

for user in User:
    items = User[user]
    for item in items:
        user_list.append(user)
        item_list.append(item[1])

In [ ]:
each_item_num = '_ItemPerUser=' + str(item_per_user) if is_item_per_user_num_const else ''
user_num_in_data= '_UserNum=' + str(user_num) if is_user_num_const else ''

fiexd_df =  pd.DataFrame(
            data={
                'user_id': user_list,
                'item_id': item_list
            }
)

In [ ]:
fiexd_df.to_csv("{}/processing_data/{}_processing{}{}.txt".format(dataset, dataset,user_num_in_data, each_item_num), header=None, index=False, sep=' ')